In [84]:
using JuMP
using Ipopt
using MadNLP
using NamedArrays

In [85]:
GU = Dict(
    :vref => 2,
    :piref => .25,
    :esub => 4,
    :thetaK => 0.3,
    :thetaE => .3,
    :thetaL => .2,
    :w => 1,
    :sigma_p => .5,
    :s => Symbol.("s",1:50),
    :wage => 1,
    :stochastic => 1,
)   

GU[:gamma] = 1- 1/GU[:esub]
GU[:pi0] = 1- GU[:thetaK] - GU[:thetaE] - GU[:thetaL]
GU[:sigma] = (log(1+GU[:pi0]) - log(1+GU[:pi0]*GU[:piref]))/log(GU[:vref])
GU[:rho] = 1-1/GU[:sigma]


GU[:p] = NamedArray(exp.(GU[:sigma_p].*rand(length(GU[:s]))),GU[:s])


X = split("s1  0.855,    s2  1.178,    s3  1.261,    s4  0.401,    s5  0.694,    s6  0.615,    s7  0.821,    s8  1.596,    s9  0.684,    s10 1.000,    s11 0.635,    s12 1.188,    s13 0.718,    s14 0.784,    s15 2.565,    s16 0.697,    s17 0.522,    s18 0.503,    s19 1.173,    s20 1.810,    s21 0.562,    s22 1.426,    s23 1.663,    s24 0.696,    s25 0.607,    s26 1.003,    s27 0.608,    s28 0.635,    s29 1.393,    s30 2.193,    s31 3.024,    s32 0.732,    s33 0.897,    s34 0.618,    s35 0.948,    s36 0.877,    s37 0.769,    s38 0.595,    s39 2.694,    s40 1.511,    s41 1.652,    s42 0.687,    s43 1.459,    s44 1.822,    s45 1.525,    s46 0.489,    s47 2.857,    s48 1.400,    s49 1.088,    s50 2.281",",")


GU[:p][GU[:s]] = [parse(Float64,e[end]) for e in split.(strip.(X)," ")]
1;

In [93]:
function gigeconomy(GU)

    S = GU[:s]# => Symbol.("s",1:50),

    vref = GU[:vref]
    piref=  GU[:piref]
    esub = GU[:esub]
    θk = GU[:thetaK]# => 0.3,
    θE = GU[:thetaE]# => .3,
    θL = GU[:thetaL]# => .2,
    w = GU[:w]# => 1,
    σp = GU[:sigma_p]# => .5,

    wage = GU[:wage]# => 1,
    stochastic = GU[:stochastic]# => 1,

    γ = GU[:gamma]# = 1- 1/GU[:esub]
    π0 = GU[:pi0] # = 1- GU[:thetaK] - GU[:thetaE] - GU[:thetaL]
    σ = GU[:sigma] # = (log(1+GU[:pi0]) - log(1+GU[:pi0]*GU[:piref]))/log(GU[:vref])
    ρ = GU[:rho] # = 1-1/GU[:sigma]


    p = GU[:p]
    
    m = Model(MadNLP.Optimizer)

    @variables(m,begin
        EU		                    #Expected utility,
        K[S]>=.01, (start = θk,)	#Capital stock,
        E[S]>=.01, (start = θE,)	#Full time employees,
        KK		                    #Capital stock (stochastic model)
        EE		                    #Employment (stochastic model)
        L[S]>=.01, (start = θL,)	#Gig employees

        X[S]>=.001, (start = 1,)	#Ending wealth by state,
        PI[S],      (start=π0,)		#Profit (or social surplus) by state,
        Y[S],       (start = 1,)	#Supply = demand by state,
        EL[S],      (start = 1,)	#Labor-employee nest
    end)
    
    @NLobjective(m,Max,EU)

    N = length(50)

    @NLconstraints(m,begin
        eudef, EU == 1/N * sum(X[s]^ρ for s∈S)^(1/ρ)
        eldef[s=S], EL[s] == (θE/(θE+θL) * ( E[s]/θE)^γ + θL/(θE+θL) * (L[s]/θL)^γ)^(1/γ)
        ydef[s=S], Y[s] == (K[s]/θk)^θk * EL[s]^(θE+θL)
        pidef[s=S], PI[s] == p[s]*Y[s] - (K[s] + E[s] + wage*L[s])
        xdef[s=S], X[s] == (w+PI[s])/(w+π0)
    end)

    if stochastic != 0
        @NLconstraints(m,begin
            naK[s=S], K[s] == KK
            naE[s=S], E[s] == EE
        end)
    end

    return m
end

gigeconomy (generic function with 1 method)

In [94]:
GU[:wage]=1
GU[:stochastic] = 1

bench = gigeconomy(GU)
set_silent(bench)
optimize!(bench)

out = ""

for var in all_variables(bench)
    out*="$var -> $(value(var))\n"
end

#println(out)

In [95]:
GU[:wage]=1
GU[:stochastic] = 0

shock = gigeconomy(GU)
set_silent(shock)
optimize!(shock)

out = ""

for var in all_variables(shock)
    out*="$var -> $(value(var))\n"
end

#println(out)


In [96]:
EU_s = value(bench[:EU])#objective_value(bench)
EU_d = sum(value.(shock[:X]))/length(GU[:s])

rp = 100*(sum(value.(bench[:X]))/length(GU[:s])/EU_s-1)
evpi = 100 * (EU_d / EU_s - 1);

println(
"""EU_s: $(EU_s) Expected utility in stochastic model
EU_d: $(EU_d) Expected utility in the deterministic model

rp  : $(rp) Risk premium (%)
evpi: $(evpi) Expected value of perfect information
""")

EU_s: 0.37029729177311527 Expected utility in stochastic model
EU_d: 3.6110454455753263 Expected utility in the deterministic model

rp  : 191.24924385735392 Risk premium (%)
evpi: 875.1746841799342 Expected value of perfect information



In [97]:
value(bench[:EU])

0.37029729177311527

In [98]:
value(shock[:EU])

0.388266106797141

In [91]:
value.(shock[:X])

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:s1, :s2, :s3, :s4, :s5, :s6, :s7, :s8, :s9, :s10  …  :s41, :s42, :s43, :s44, :s45, :s46, :s47, :s48, :s49, :s50]
And data, a 50-element Vector{Float64}:
  0.9094849842940702
  1.2114059315174424
  1.3647365543086578
  0.8350614401023079
  0.860164904349237
  0.8479963837633891
  0.895500995397868
  2.559223613942403
  0.8582867063154686
  0.99999999999597
  ⋮
  0.8588387507096928
  1.935190266901405
  4.179831105645636
  2.208001790184187
  0.8379934097924047
 27.467882351362515
  1.7297066664466711
  1.0874266408269015
 11.120312705056179